In [ ]:
!pip install -U "transformers>=4.44.0" accelerate huggingface_hub
!pip install -U sentencepiece

  Using cached huggingface_hub-1.2.1-py3-none-any.whl.metadata (13 kB)


In [ ]:
import pandas as pd
from getpass import getpass
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import torch
import threading
from tqdm import tqdm
import json
import time
from datetime import datetime

print("Loading descriptions dataset...")
descriptions = pd.read_parquet("hf://datasets/lang-uk/recruitment-dataset-job-descriptions-english/data/train-00000-of-00001.parquet")
print("Descriptions head:")
print(descriptions.head())
print("-" * 30)

print("Loading profiles dataset...")
profile = pd.read_parquet("hf://datasets/lang-uk/recruitment-dataset-candidate-profiles-english/data/train-00000-of-00001.parquet")
print("Profiles head:")
print(profile.head())
print("-" * 30)

print("Logging in to Hugging Face Hub...")
try:
    HF_TOKEN = getpass()
    login(HF_TOKEN)
    print("Login successful.")
except Exception as e:
    print(f"Login failed: {e}")
print("-" * 30)
MODELS_TO_TEST = {
    # "gemma-2-2b-it": "google/gemma-2-2b-it",
    # "gemma-2-9b-it": "google/gemma-2-9b-it",
    # "gemma-2-27b-it": "google/gemma-2-27b-it",

    # "qwen2-0.5b-instruct": "Qwen/Qwen2-0.5B-Instruct",
    # "qwen2-1.5b-instruct": "Qwen/Qwen2-1.5B-Instruct",
    # "qwen2-7b-instruct":   "Qwen/Qwen2-7B-Instruct",
    # "qwen2-72b-instruct":  "Qwen/Qwen2-72B-Instruct",

    # "mistral-7b-instruct": "mistralai/Mistral-7B-Instruct-v0.3",
    # "mistral-nemo-12b":    "mistralai/Mistral-Nemo-Instruct-2407",
    # "mixtral-8x7b-instruct": "mistralai/Mixtral-8x7B-Instruct-v0.1",

    # "phi-3-mini-4k-instruct":   "microsoft/Phi-3-mini-4k-instruct",
    # "phi-3-small-8k-instruct":  "microsoft/Phi-3-small-8k-instruct",
    "phi-3-medium-4k-instruct": "microsoft/Phi-3-medium-4k-instruct"
}


Loading descriptions dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Descriptions head:
                                            Position  \
0      10 + Blockchain Nodes / Masternodes to set up   
1       10 .NET Developers (Middle and Senior level)   
2  10X Engineer (co-founder, #4 employee, USD 11-...   
3                          16 - Amazon Brand Manager   
4                          16 - Amazon Brand Manager   

                                    Long Description    Company Name  \
0  *Requirements*\r\n\r\nWe're looking for a long...    MyCointainer   
1  Greetings! My name is Maria, I am in urgent ne...  TechScout.tech   
2  **Product**\r\nThe product is a live video cha...        Innoteka   
3  Currently, TCM expanding its activities to Ukr...       FirstFive   
4  Hello,\r\nWe, MIMIRB2B, are an outstaff compan...        MimirB2B   

  Exp Years Primary Keyword English Level                  Published  \
0        2y        Sysadmin  intermediate  2020-10-01T00:00:00+03:00   
1        2y            .NET  intermediate  2022-03-01T00:00:00+02:0

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    print("Using CPU - це буде повільно!")

print("-" * 30)


Using GPU: Tesla T4
GPU memory: 14.7 GB
------------------------------


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


In [ ]:

def generate_resumes_for_model(model_name, model_path, prompts, device):
    print(f"\n{'='*50}")
    print(f"Loading {model_name}...")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        ).eval()

        tokenizer = AutoTokenizer.from_pretrained(model_path)

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        print(f"Model {model_name} loaded successfully")

        generated_resumes = []
        generation_times = []

        test_prompts = prompts[:1]

        for i, prompt_text in enumerate(tqdm(test_prompts, desc=f"Generating with {model_name}")):
            start_time = time.time()

            try:
                messages = [{"role": "user", "content": prompt_text}]

                if hasattr(tokenizer, 'apply_chat_template'):
                    try:
                        prompt_with_template = tokenizer.apply_chat_template(
                            messages, tokenize=False, add_generation_prompt=True
                        )
                    except:
                        prompt_with_template = prompt_text
                else:
                    prompt_with_template = prompt_text

                inputs = tokenizer(prompt_with_template, return_tensors="pt", truncation=True, max_length=2048)
                inputs = {k: v.to(device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=512,
                        temperature=0.7,
                        top_p=0.9,
                        do_sample=True,
                        eos_token_id=tokenizer.eos_token_id,
                        pad_token_id=tokenizer.pad_token_id,
                        repetition_penalty=1.1
                    )

                resume_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

                end_time = time.time()
                generation_time = end_time - start_time

                generated_resumes.append({
                    "candidate_id": i,
                    "resume": resume_text,
                    "generation_time_seconds": generation_time,
                    "model": model_name
                })

                generation_times.append(generation_time)

                print(f"\n{model_name} - Candidate {i+1}: {generation_time:.2f}s")
                print(f"Output length: {len(resume_text)} characters")

            except Exception as e:
                print(f"Error generating for candidate {i+1}: {e}")
                generated_resumes.append({
                    "candidate_id": i,
                    "resume": f"ERROR: {str(e)}",
                    "generation_time_seconds": 0,
                    "model": model_name,
                    "error": str(e)
                })

        del model
        del tokenizer
        if device == "cuda":
            torch.cuda.empty_cache()

        return {
            "model_name": model_name,
            "model_path": model_path,
            "resumes": generated_resumes,
            "avg_generation_time": sum(generation_times) / len(generation_times) if generation_times else 0,
            "total_time": sum(generation_times),
            "status": "success"
        }

    except Exception as e:
        print(f"Failed to load {model_name}: {e}")
        return {
            "model_name": model_name,
            "model_path": model_path,
            "resumes": [],
            "error": str(e),
            "status": "failed"
        }



In [ ]:

print("Preparing prompts...")
prompt_style = """Below is an instruction that describes a task, paired with an input that provides candidate details and a target job.
Write a professional, ATS-friendly resume tailored to the target role.

First, produce a concise 2–4 bullet **Plan** that lists the sections and focus points you will include (e.g., highlight leadership, quantify achievements, include keywords from job description). Do not reveal internal chain-of-thought — keep the plan explicit and short.

Then generate the resume. Use clear section headers (Summary, Experience, Education, Skills, Projects/Certs as applicable). For experience bullets, use the STAR/impact style (situation/task → action → measurable result) and include quantifiable metrics where possible. Tailor language and keywords to the target job.

### Candidate details / Job target:
{}

### Additional instructions (tone, length, must-include keywords, formatting notes):
{}

### Output format:
Plan:
- <short bullet 1>
- <short bullet 2>

Resume:
[Use sections: Summary, Experience (most recent first), Education, Skills, Projects/Certifications, Additional information (optional)]
"""

complex_cot = (
    "- Identify key skills from the candidate's past roles.\n"
    "- Match these skills to the job description keywords.\n"
    "- Prioritize experiences that show measurable achievements."
)

def format_prompts(df, extra_instructions="Tone: professional, one-page, include relevant keywords."):
    candidate_details = [
        f"Position: {p}\nMore info: {m}\nLooking For: {l}\nHighlights: {h}\nPrimary Keyword: {k}"
        for p, m, l, h, k in zip(
            df["Position"],
            df["Moreinfo"],
            df["Looking For"],
            df["Highlights"],
            df["Primary Keyword"]
        )
    ]

    prompts = [
        prompt_style.format(details, extra_instructions + "\n" + complex_cot)
        for details in candidate_details
    ]
    return prompts

test_profiles = profile.head(1)
prompts = format_prompts(test_profiles)
print(f"Prepared {len(prompts)} prompts for testing")
print("-" * 30)



Preparing prompts...
Prepared 1 prompts for testing
------------------------------


In [ ]:

all_results = {
    "test_config": {
        "device": device,
        "test_date": datetime.now().isoformat(),
        "number_of_candidates": 1,
        "max_new_tokens": 1500,
        "total_profiles_available": len(profile),
        "profiles_used": len(test_profiles)
    },
    "model_results": []
}

print("Starting model comparison benchmark...")

for model_name, model_path in MODELS_TO_TEST.items():
    result = generate_resumes_for_model(model_name, model_path, prompts, device)
    all_results["model_results"].append(result)

    if result["status"] == "success":
        print(f"✅ {model_name}: {result['avg_generation_time']:.2f}s avg per resume")
    else:
        print(f"❌ {model_name}: Failed - {result.get('error', 'Unknown error')}")



Starting model comparison benchmark...

Loading phi-3-medium-4k-instruct...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Model phi-3-medium-4k-instruct loaded successfully


Generating with phi-3-medium-4k-instruct: 100%|██████████| 1/1 [00:00<00:00,  5.32it/s]

Error generating for candidate 1: 'DynamicCache' object has no attribute 'seen_tokens'
✅ phi-3-medium-4k-instruct: 0.00s avg per resume


In [ ]:

output_filename = f"resume_generation_benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print(f"\n{'='*50}")
print("Benchmark completed! Results saved to:", output_filename)



Benchmark completed! Results saved to: resume_generation_benchmark_20251206_224054.json


In [ ]:

print("\nPERFORMANCE SUMMARY:")
print("=" * 50)
successful_models = [r for r in all_results["model_results"] if r["status"] == "success"]
failed_models = [r for r in all_results["model_results"] if r["status"] == "failed"]

if successful_models:
    print("SUCCESSFUL MODELS:")
    for result in successful_models:
        print(f"  {result['model_name']:25} | {result['avg_generation_time']:6.2f}s avg | {result['total_time']:6.2f}s total")

if failed_models:
    print("\nFAILED MODELS:")
    for result in failed_models:
        print(f"  {result['model_name']:25} | {result.get('error', 'Unknown error')}")

print(f"\nDevice used: {device}")
if device == "cuda":
    print(f"GPU memory allocated: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")

print(f"\nSAMPLE RESULTS:")
print("=" * 50)
for result in successful_models:
    if result["resumes"]:
        first_resume = result["resumes"][0]["resume"]
        preview = first_resume[:200] + "..." if len(first_resume) > 200 else first_resume
        print(f"\n{result['model_name']} - First resume preview:")
        print(preview)
        print("-" * 30)


PERFORMANCE SUMMARY:
SUCCESSFUL MODELS:
  phi-3-medium-4k-instruct  |   0.00s avg |   0.00s total

Device used: cuda
GPU memory allocated: 12.37 GB

SAMPLE RESULTS:

phi-3-medium-4k-instruct - First resume preview:
ERROR: 'DynamicCache' object has no attribute 'seen_tokens'
------------------------------
